#Imports and loading model

In [ ]:
import torch
from transformers import T5TokenizerFast as T5Tokenizer, T5ForConditionalGeneration
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/MyDrive/Project/Text Summarizer/ModelT5Base"

if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)
    print(f"Created directory: {DRIVE_PATH}")
else:
    print(f"Directory exists: {DRIVE_PATH}")

Mounted at /content/drive
Directory exists: /content/drive/MyDrive/Project/Text Summarizer/ModelT5Base


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Loading model from {DRIVE_PATH}...")
model = T5ForConditionalGeneration.from_pretrained(DRIVE_PATH).to(device)
tokenizer = T5Tokenizer.from_pretrained(DRIVE_PATH)
print("Model loaded successfully!")

Loading model from /content/drive/MyDrive/Project/Text Summarizer/ModelT5Base...
Model loaded successfully!


#preparing logic

In [ ]:
def generate_summary(text, style, model, tokenizer):
    model.eval()
    input_text = f"summarize {style}: {text}"

    input_words = len(text.split())

    inputs = tokenizer(
        input_text,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    ngram_size = 3

    if style == "harsh":
        max_len = int(input_words * 0.35)
        min_len = 10
        len_penalty = 1.0
        rep_penalty = 2.0

    elif style == "standard":
        max_len = 150
        min_len = 60
        len_penalty = 2.0
        rep_penalty = 1.5

    elif style == "detailed":
        if input_words < 150:
            max_len = 300
            min_len = int(input_words * 0.85)
            len_penalty = 3.0
            rep_penalty = 1.1
            ngram_size = 3
        else:
            max_len = int(input_words * 1.5)
            min_len = int(input_words * 0.5)
            len_penalty = 1.5
            rep_penalty = 1.2
            ngram_size = 4

    # Safety caps
    max_len = min(max_len, 512)
    min_len = min(min_len, 200)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_len,
            min_length=min_len,
            length_penalty=len_penalty,
            num_beams=4,
            repetition_penalty=rep_penalty,
            no_repeat_ngram_size=ngram_size,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

#summarize test

In [ ]:
text = """
    Russian President Vladimir Putin said that a US plan to end the war in Ukraine could “form the basis for future agreements”
    but renewed threats to seize more territory by force unless Kyiv withdraws. Speaking to reporters in Bishtek, in the Central
    Asian republic of Kyrgyzstan on Thursday, Putin confirmed the Kremlin was expecting a US delegation headed by Special Envoy
    Steve Witkoff to visit Moscow early next week, adding that the Kremlin was ready for “serious discussion”. But chances of a
    swift breakthrough appear slim after Putin repeated his maximalist demands, saying the war in Ukraine will only end “once
    Ukrainian troops withdraw from the territories they occupy.” “If they don’t withdraw, we will achieve this through military
    means,” the Russian leader said. Russia is occupying some 20% of the territory recognized under international law as part of
    sovereign Ukraine, including almost all of the Luhansk region, and parts of the Donetsk, Kherson and Zaporizhzhia regions.
    Moscow is demanding Ukraine surrender the entirety of these four regions, which it has annexed but not fully conquered. Russia
    has made some gains along the eastern Ukrainian frontline in recent weeks, most significantly around the city of Pokrovsk. Still,
    the Institute for the Study of War, a US-based conflict monitor, said on Thursday that data on Russian forces’ rate of advance
    indicates that “a Russian military victory in Ukraine is not inevitable, and a rapid Russian seizure of the rest of Donetsk Oblast
    is not imminent.” Crucially, the area Russia is demanding includes the “fortress belt” of heavily defended towns and cities seen
    that are seen as vital for Ukrainian security. Kyiv and its European allies have made it clear that territorial concessions are a red line
    for them. Putin’s remarks on Thursday were the strongest indication that Russia is not willing to move after US officials, including
    Trump himself, touted “tremendous progress” being made on their efforts to end the war. This came after Ukrainian and European
    officials strongly opposed and then revised the 28-point peace plan which was drafted by the US with apparent strong input from Russia.
    The original plan reflected Russia’s extensive wishlist, and included a demand for Ukraine to reduce its army and be barred from
    joining NATO. Putin said on Thursday that he was expecting Witkoff to arrive to Moscow early next week, presumably to discuss the
    new draft of the plan, the exact wording of which is not yet known. Putin said he has been informed of the latest discussions and that
    the plan could “form the basis for future agreements.” “It would be impolite of me to speak of final agreements now,” Putin added.
"""

In [ ]:
print(f"Original Word Count: {len(text.split())}")
print(text)
print("-" * 50)

# 1. Harsh
harsh_out = generate_summary(text, "harsh", model, tokenizer)
print(f"\n--- HARSH (Short & Direct) ---")
print(harsh_out)
print(f"Word Count: {len(harsh_out.split())}")

# 2. Standard
standard_out = generate_summary(text, "standard", model, tokenizer)
print(f"\n--- STANDARD (Balanced) ---")
print(standard_out)
print(f"Word Count: {len(standard_out.split())}")

# 3. Detailed
detailed_out = generate_summary(text, "detailed", model, tokenizer)
print(f"\n--- DETAILED (Comprehensive) ---")
print(detailed_out)
print(f"Word Count: {len(detailed_out.split())}")

Original Word Count: 440

    Russian President Vladimir Putin said that a US plan to end the war in Ukraine could “form the basis for future agreements”
    but renewed threats to seize more territory by force unless Kyiv withdraws. Speaking to reporters in Bishtek, in the Central
    Asian republic of Kyrgyzstan on Thursday, Putin confirmed the Kremlin was expecting a US delegation headed by Special Envoy
    Steve Witkoff to visit Moscow early next week, adding that the Kremlin was ready for “serious discussion”. But chances of a
    swift breakthrough appear slim after Putin repeated his maximalist demands, saying the war in Ukraine will only end “once
    Ukrainian troops withdraw from the territories they occupy.” “If they don’t withdraw, we will achieve this through military
    means,” the Russian leader said. Russia is occupying some 20% of the territory recognized under international law as part of
    sovereign Ukraine, including almost all of the Luhansk region, and parts o

In [ ]:
text_to_summarize = """
    The James Webb Space Telescope (JWST) has captured a lush landscape of stellar birth.
    The new image shows the Cosmic Cliffs, which are the edge of a giant gaseous cavity within the star-forming region NGC 3324.
    The cavity has been carved from the nebula by the intense ultraviolet radiation and stellar winds from extremely massive, hot, young stars located in the center of the bubble, above the area shown in this image.
    The high peaks of the "cliffs" are about 7 light-years high.
    The blistering radiation from the young stars is sculpting the nebula's wall by slowly eroding it away.
    Simultaneously, the same gravity that pulls matter together to form stars also tears them apart in violent outbursts.
"""

print(f"Original Word Count: {len(text_to_summarize.split())}")
print(text_to_summarize)
print("-" * 50)

# 1. Harsh
harsh_out = generate_summary(text_to_summarize, "harsh", model, tokenizer)
print(f"\n--- HARSH (Short & Direct) ---")
print(harsh_out)
print(f"Word Count: {len(harsh_out.split())}")

# 2. Standard
standard_out = generate_summary(text_to_summarize, "standard", model, tokenizer)
print(f"\n--- STANDARD (Balanced) ---")
print(standard_out)
print(f"Word Count: {len(standard_out.split())}")

# 3. Detailed
detailed_out = generate_summary(text_to_summarize, "detailed", model, tokenizer)
print(f"\n--- DETAILED (Comprehensive) ---")
print(detailed_out)
print(f"Word Count: {len(detailed_out.split())}")

Original Word Count: 118

    The James Webb Space Telescope (JWST) has captured a lush landscape of stellar birth.
    The new image shows the Cosmic Cliffs, which are the edge of a giant gaseous cavity within the star-forming region NGC 3324.
    The cavity has been carved from the nebula by the intense ultraviolet radiation and stellar winds from extremely massive, hot, young stars located in the center of the bubble, above the area shown in this image.
    The high peaks of the "cliffs" are about 7 light-years high.
    The blistering radiation from the young stars is sculpting the nebula's wall by slowly eroding it away.
    Simultaneously, the same gravity that pulls matter together to form stars also tears them apart in violent outbursts.

--------------------------------------------------

--- HARSH (Short & Direct) ---
The new image shows the Cosmic Cliffs, which are the edge of a giant gaseous cavity within the star-forming region NGC 3324.
Word Count: 22

--- STANDARD (Balan

#upload to hugging face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
# Paste your token when asked

In [ ]:
import torch
from transformers import T5TokenizerFast as T5Tokenizer, T5ForConditionalGeneration

# 1. Define your Google Drive Path (Where your trained model is)
DRIVE_PATH = "/content/drive/MyDrive/Project/Text Summarizer/ModelT5Base"

# 2. Load the model locally first
print(f"Loading model from {DRIVE_PATH}...")
model = T5ForConditionalGeneration.from_pretrained(DRIVE_PATH)
tokenizer = T5Tokenizer.from_pretrained(DRIVE_PATH)

# 3. Push to Hugging Face
# Replace 'your-username' and 'model-name' with whatever you want
repo_name = "Hiratax/t5-news-summarizer"

print(f"Pushing to Hugging Face: {repo_name}...")
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print("Upload Complete! Check your profile on Hugging Face.")

Loading model from /content/drive/MyDrive/Project/Text Summarizer/ModelT5Base...
Pushing to Hugging Face: Hiratax/t5-news-summarizer...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...5594w7y/model.safetensors:   1%|          | 9.40MB /  990MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../tmpw6dcb1pi/spiece.model: 100%|##########|  792kB /  792kB            

Upload Complete! Check your profile on Hugging Face.


#load from hugging face

In [ ]:
from transformers import T5TokenizerFast as T5Tokenizer, T5ForConditionalGeneration
import torch

# 1. Your Hugging Face Repo ID
# (Make sure this matches the one you uploaded to)
HUB_MODEL_ID = "Hiratax/t5-news-summarizer"

# 2. Load directly from the Hub
print(f"Downloading model from Hugging Face: {HUB_MODEL_ID}...")
tokenizerHG = T5Tokenizer.from_pretrained(HUB_MODEL_ID)
modelHG = T5ForConditionalGeneration.from_pretrained(HUB_MODEL_ID)

# 3. Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelHG.to(device)

print("Model loaded successfully from Hugging Face!")

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Model loaded successfully from Hugging Face!


In [ ]:
print(f"Original Word Count: {len(text.split())}")
print(text)
print("-" * 50)

# 1. Harsh
harsh_out = generate_summary(text, "harsh", modelHG, tokenizerHG)
print(f"\n--- HARSH (Short & Direct) ---")
print(harsh_out)
print(f"Word Count: {len(harsh_out.split())}")

# 2. Standard
standard_out = generate_summary(text, "standard", modelHG, tokenizerHG)
print(f"\n--- STANDARD (Balanced) ---")
print(standard_out)
print(f"Word Count: {len(standard_out.split())}")

# 3. Detailed
detailed_out = generate_summary(text, "detailed", modelHG, tokenizerHG)
print(f"\n--- DETAILED (Comprehensive) ---")
print(detailed_out)
print(f"Word Count: {len(detailed_out.split())}")

Original Word Count: 440

    Russian President Vladimir Putin said that a US plan to end the war in Ukraine could “form the basis for future agreements”
    but renewed threats to seize more territory by force unless Kyiv withdraws. Speaking to reporters in Bishtek, in the Central
    Asian republic of Kyrgyzstan on Thursday, Putin confirmed the Kremlin was expecting a US delegation headed by Special Envoy
    Steve Witkoff to visit Moscow early next week, adding that the Kremlin was ready for “serious discussion”. But chances of a
    swift breakthrough appear slim after Putin repeated his maximalist demands, saying the war in Ukraine will only end “once
    Ukrainian troops withdraw from the territories they occupy.” “If they don’t withdraw, we will achieve this through military
    means,” the Russian leader said. Russia is occupying some 20% of the territory recognized under international law as part of
    sovereign Ukraine, including almost all of the Luhansk region, and parts o

In [ ]:
print(f"Original Word Count: {len(text_to_summarize.split())}")
print(text_to_summarize)
print("-" * 50)

# 1. Harsh
harsh_out = generate_summary(text_to_summarize, "harsh", modelHG, tokenizerHG)
print(f"\n--- HARSH (Short & Direct) ---")
print(harsh_out)
print(f"Word Count: {len(harsh_out.split())}")

# 2. Standard
standard_out = generate_summary(text_to_summarize, "standard", modelHG, tokenizerHG)
print(f"\n--- STANDARD (Balanced) ---")
print(standard_out)
print(f"Word Count: {len(standard_out.split())}")

# 3. Detailed
detailed_out = generate_summary(text_to_summarize, "detailed", modelHG, tokenizerHG)
print(f"\n--- DETAILED (Comprehensive) ---")
print(detailed_out)
print(f"Word Count: {len(detailed_out.split())}")

Original Word Count: 118

    The James Webb Space Telescope (JWST) has captured a lush landscape of stellar birth.
    The new image shows the Cosmic Cliffs, which are the edge of a giant gaseous cavity within the star-forming region NGC 3324.
    The cavity has been carved from the nebula by the intense ultraviolet radiation and stellar winds from extremely massive, hot, young stars located in the center of the bubble, above the area shown in this image.
    The high peaks of the "cliffs" are about 7 light-years high.
    The blistering radiation from the young stars is sculpting the nebula's wall by slowly eroding it away.
    Simultaneously, the same gravity that pulls matter together to form stars also tears them apart in violent outbursts.

--------------------------------------------------

--- HARSH (Short & Direct) ---
The new image shows the Cosmic Cliffs, which are the edge of a giant gaseous cavity within the star-forming region NGC 3324.
Word Count: 22

--- STANDARD (Balan

In [ ]:
t = "Russian President Vladimir Putin says war in Ukraine will end “once Ukrainian troops withdraw from the territories they occupy” but insists it will only end once Kyiv withdraws from the territory it is occupying. The Kremlin is expecting a US delegation headed by Special Envoy Steve Witkoff to visit Moscow early next week, according to reports. But chances of a swift breakthrough appear slim after Putin reiterated his maximalist demands, saying the war will end ‘once Ukrainian forces withdraw from their territories’. He also said he was ready for ‘serious discussion’ and that the plan could ‘form the basis for future agreements’. Russia has made gains along the eastern Ukrainian frontline in recent weeks, most significantly around the city of Pokrovsk. However, data on Russian forces’ rate of advance indicates that ‘a Russian military victory in Ukraine is not inevitable, and a rapid Russian seizure of the rest of Donetsk Oblast is not imminent’."

print(len(t.split()))

154
